<a href="https://colab.research.google.com/github/lifewinning/MLTSA22_final_IBHPMP/blob/main/data_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## Initialize EarthEngine, import geemap



In [1]:
import ee
 
# Trigger the authentication flow.
ee.Authenticate()
 
# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=vqmB-CQjZwqosymzU1CHi8Q-KoOcd1dBnLRKcZ-os1g&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWhDDs6nxYfYIDHml-A0EFRiZih3F5CMGx0kVZ8Qn8E4Q1Sm_NcW0rA

Successfully saved authorization token.


In [2]:
pip install geemap

In [3]:
import geemap
Map = geemap.Map()

In [4]:
import json

# Mount Drive

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
cd /content/drive/MyDrive/MLTSA22

/content/drive/MyDrive/MLTSA22


# Set date params and load clipping features

In [122]:
# establish time series window
start = ee.Date('2018-01-01')
finish = ee.Date('2021-12-31')

In [144]:
# load a geojson with bounding boxes
with open('mltsa_bb.geojson', 'r') as file:
     geojson = json.load(file)
for site, index in enumerate(range(0,4)):
  print(index, geojson['features'][site]['properties']['site'])


0 singapore
1 suez
2 long beach
3 rotterdam


# Some generalizing functions

In [9]:
def cliptoaoi(image):
  return image.clip(aoi)

In [60]:
# generate a range of days between start and end of time series
def getdates(collection):
  first = collection.first().date()
  print(first.format('yyy-MM-dd').getInfo())
  diff = finish.difference(first, 'day')
  def getDays(day):
    return first.advance(day,'day')
  
  return ee.List.sequence(0, diff.subtract(1), 6).map(getDays)

In [31]:
def day_mosaics(date, newlist):
  date = ee.Date(date)
  newlist = ee.List(newlist)
  
  filtered = collection_clipped.filterDate(date, date.advance(1,'day'));

  date_formatted = date.format('yyyy-MM-dd')

  image_m = ee.Image(filtered.mosaic()).set({'ID': date_formatted}).set({'date': date}).clip(aoi)

  # Add the mosaic to a list only if the collection has images
  return ee.List(ee.Algorithms.If(filtered.size(), newlist.add(image_m), newlist))

# Suez Canal

In [23]:
coords = geojson['features'][1]['geometry']['coordinates']
aoi = ee.Geometry.Polygon(coords)

# query EE
collection = ee.ImageCollection('COPERNICUS/S1_GRD').filterBounds(aoi).filterDate(start, finish).filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
collection_clipped = collection.map(cliptoaoi).sort('segmentStartTime')
collection_list = collection_clipped.toList(collection.size())

# mosaic images from the same day as needed
newcol = ee.ImageCollection(ee.List(getdates(collection).iterate(day_mosaics, ee.List([]))))
newcol_list = newcol.toList(newcol.size())
ids = newcol.aggregate_array('ID').getInfo()

# this is for exporting
vis_params = {'min': -20, 'max': 0, 'bands':'VH'}

In [161]:
# take a quick look at the first image just to see
image = newcol.first()
out_img = "tests/suez-first.png"
geemap.get_image_thumbnail(image, out_img, vis_params, dimensions=500, format='png')
geemap.show_image(out_img)

Output()

In [163]:
out_dir = 'finalimages/suez'
geemap.get_image_collection_thumbnails(newcol, out_dir, vis_params, dimensions=500, format="png", names=ids)

Total number of images: 242

An error occurred while downloading.
Earth Engine capacity exceeded.


# Long Beach

In [99]:
# update to long beach
coords = geojson['features'][2]['geometry']['coordinates']
aoi = ee.Geometry.Polygon(coords)

# query EE
collection = ee.ImageCollection('COPERNICUS/S1_GRD').filterBounds(aoi).filterDate(start, finish).filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
collection_clipped = collection.map(cliptoaoi).sort('segmentStartTime')
collection_list = collection_clipped.toList(collection_clipped.size())

In [65]:
# generate a range of days between start and end of time series
def getdates(coll):
  first = coll.first().date()
  diff = finish.difference(first, 'day')
  def getDays(day):
    return first.advance(day,'day')
  
  return ee.List.sequence(0, diff.subtract(1), 6).map(getDays)

In [67]:
# mosaic images from the same day as needed, make list of IDs from mosaic images
newcol = ee.ImageCollection(ee.List(getdates(collection_clipped).iterate(day_mosaics, ee.List([]))))
newcol_list = newcol.toList(newcol.size())
ids = newcol.aggregate_array('ID').getInfo()

# this is for exporting
vis_params = {'min': -20, 'max': 0, 'bands':'VH'}

In [70]:
# take a quick look at the first image just to see
out_img = "tests/longbeach-first.png"
image = newcol.first()
geemap.get_image_thumbnail(image, out_img, vis_params, dimensions=500, format='png')
geemap.show_image(out_img)

Output()

In [71]:
out_dir = 'finalimages/longbeach'
geemap.get_image_collection_thumbnails(newcol, out_dir, vis_params, dimensions=500, format="png", names=ids)

Total number of images: 222



# Rotterdam

In [145]:
# generate a range of days between start and end of time series
def getdates(coll):
  diff = finish.difference(start, 'day')
  def getDays(day):
    return start.advance(day,'day')
  
  return ee.List.sequence(0, diff.subtract(1),6).map(getDays)

In [146]:
# update to rotterdam
coords = geojson['features'][3]['geometry']['coordinates']
aoi = ee.Geometry.Polygon(coords)

# query EE
collection = ee.ImageCollection('COPERNICUS/S1_GRD').filterBounds(aoi).filterDate(start, finish).filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')).filter(ee.Filter.eq('instrumentMode', 'IW')).filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))
collection_clipped = collection.map(cliptoaoi).sort('segmentStartTime')
collection_list = collection_clipped.toList(collection_clipped.size())

# mosaic images from the same day as needed, make list of IDs from mosaic images
newcol = ee.ImageCollection(ee.List(getdates(collection_clipped).iterate(day_mosaics, ee.List([]))))
newcol_list = newcol.toList(newcol.size())
ids = newcol.aggregate_array('ID').getInfo()

# this is for exporting
vis_params = {'min': -20, 'max': 0, 'bands':'VH'}

In [147]:
len(ids), ids

(239,
 ['2018-01-01',
  '2018-01-07',
  '2018-01-13',
  '2018-01-19',
  '2018-01-25',
  '2018-01-31',
  '2018-02-06',
  '2018-02-12',
  '2018-02-18',
  '2018-02-24',
  '2018-03-02',
  '2018-03-08',
  '2018-03-14',
  '2018-03-20',
  '2018-03-26',
  '2018-04-01',
  '2018-04-07',
  '2018-04-13',
  '2018-04-19',
  '2018-04-25',
  '2018-05-01',
  '2018-05-07',
  '2018-05-13',
  '2018-05-19',
  '2018-05-25',
  '2018-05-31',
  '2018-06-06',
  '2018-06-12',
  '2018-06-18',
  '2018-06-24',
  '2018-06-30',
  '2018-07-06',
  '2018-07-12',
  '2018-07-18',
  '2018-07-24',
  '2018-07-30',
  '2018-08-05',
  '2018-08-11',
  '2018-08-17',
  '2018-08-23',
  '2018-09-04',
  '2018-09-10',
  '2018-09-16',
  '2018-09-22',
  '2018-09-28',
  '2018-10-04',
  '2018-10-10',
  '2018-10-16',
  '2018-10-22',
  '2018-10-28',
  '2018-11-03',
  '2018-11-09',
  '2018-11-15',
  '2018-11-21',
  '2018-11-27',
  '2018-12-03',
  '2018-12-09',
  '2018-12-15',
  '2018-12-21',
  '2018-12-27',
  '2019-01-02',
  '2019-01-08',
  

In [148]:
# take a quick look at the first image just to see
out_img = "tests/rotterdam-first.png"
image = newcol.first()
geemap.get_image_thumbnail(image, out_img, vis_params, dimensions=500, format='png')
geemap.show_image(out_img)

Output()

In [149]:
out_dir = 'finalimages/rotterdam'
geemap.get_image_collection_thumbnails(newcol, out_dir, vis_params, dimensions=500, format="png", names=ids)

Total number of images: 239



# Singapore

In [131]:
# update to singapore
coords = geojson['features'][0]['geometry']['coordinates']
aoi = ee.Geometry.Polygon(coords)

# query EE
collection = ee.ImageCollection('COPERNICUS/S1_GRD').filterBounds(aoi).filterDate(start, finish).filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
collection_clipped = collection.map(cliptoaoi).sort('segmentStartTime')
collection_list = collection_clipped.toList(collection_clipped.size())

In [143]:
# generate a range of days between start and end of time series
# because of the size of the image and gaps in overlap days (?) Singapore actually has a 12 day lag
def getdates(coll):
  first = ee.Date('2018-01-06')
  diff = finish.difference(first, 'day')
  def getDays(day):
    return first.advance(day,'day')
  
  return ee.List.sequence(0, diff.subtract(1),12).map(getDays)

In [140]:
# mosaic images from the same day as needed, make list of IDs from mosaic images
newcol = ee.ImageCollection(ee.List(getdates(collection_clipped).iterate(day_mosaics, ee.List([]))))
newcol_list = newcol.toList(newcol.size())
ids = newcol.aggregate_array('ID').getInfo()

# this is for exporting
vis_params = {'min': -20, 'max': 0, 'bands':'VH'}

In [136]:
# take a quick look at the first image just to see
out_img = "tests/singapore-first.png"
image = newcol.first()
geemap.get_image_thumbnail(image, out_img, vis_params, dimensions=800, format='png')
geemap.show_image(out_img)

Output()

In [142]:
out_dir = 'finalimages/singapore'
geemap.get_image_collection_thumbnails(newcol, out_dir, vis_params, dimensions=800, format="png", names=ids)

Total number of images: 118

